In [3]:
import math
import numpy as np
import random as rd 
import cv2 as cv
import os

def image_none(img_path):
    image=cv.imread(img_path)
    return image

def show_image(image):
    cv.imshow('image', image) 
    cv.waitKey(0)
    cv.destroyAllWindows()

def euclidean_distance(node1, node2):
    return math.sqrt((node1[0] - node2[0])**2 + (node1[1] - node2[1])**2)

def manhattan_distance(node1, node2):
    return abs(node1[0] - node2[0]) + abs(node1[1] - node2[1])
  
def coloring_pixel(image,color,pixel):
    if image is  not None:
        image[pixel[0],pixel[1]]=color
        return image

def define_start_goal(image,start,goal):
    start_color = [255, 0, 102] #pink 
    goal_color =  [0, 0, 204] #blue
    if image is not None:
        # should not be obstacle 
        if np.array_equal(image[start[0],start[1]],[229, 229, 229]) and np.array_equal(image[goal[0],goal[1]],[229, 229, 229]):
            image = coloring_pixel(image,start_color,start)
            image = coloring_pixel(image,goal_color,goal) 
            return image
        else: 
            print("Goal or Start is an obstacle, Redefine it!")
            return image

def draw_path(img_path,path):    
    image=cv.imread(img_path)
    if image is not None:
        for tuple in path:
            print(tuple)
            image[tuple[1],tuple[0]]=[0, 0, 255] #red
    return image

def save_image(img_path,path,image_name):
    image=cv.imread(img_path)
    new_image = draw_path(image,path)
    saving_path = '/content/'
    cv.imwrite(os.path.join(saving_path , image_name), new_image)    

def isValid(img_path,node):
    image=cv.imread(img_path,0)
    if image is not None:
        return  image[node[0],node[1]] == 229 



def imageToMatrix(img_path):
    image=cv.imread(img_path,0)
    rows,cols=(1073,1073)    
    # image (grey scale) to a matrix of 0 and 1 
    for i in range(rows):
        for j in range(cols):
            if image[i,j] == 0 or image[i,j] == 99: 
                image[i,j] = 0 #obstacle
            elif image[i,j] == 229:
                image[i,j] = 1
    return image                
    


In [5]:


def findNeighbors(matrix,node):
    #check validity and return neighbors to choose randomly one of them
    neighbors=[]

    if matrix[node[0]+1,node[1]] == 1:
        neighbor = (node[0]+1,node[1])
        neighbors.append(neighbor)

    if matrix[node[0],node[1]+1]==1:
        neighbor = (node[0],node[1]+1)
        neighbors.append(neighbor)

    if matrix[node[0]+1,node[1]+1]==1:
        neighbor = (node[0]+1,node[1]+1)
        neighbors.append(neighbor)

    if matrix[node[0]-1,node[1]]==1:
        neighbor = (node[0]-1,node[1])
        neighbors.append(neighbor)
    
    if matrix[node[0],node[1]-1]==1:
        neighbor = (node[0],node[1]-1)
        neighbors.append(neighbor)

    if matrix[node[0]-1,node[1]-1]==1:
        neighbor = (node[0]-1,node[1]-1)
        neighbors.append(neighbor)

    if matrix[node[0]-1,node[1]+1]==1:
        neighbor = (node[0]-1,node[1]+1)
        neighbors.append(neighbor)

    if matrix[node[0]+1,node[1]-1]==1:
        neighbor = (node[0]+1,node[1]-1)
        neighbors.append(neighbor) 

    return neighbors


def generatePath(matrix,start,end): 
    path=[]
    visited_nodes=[]
    path.append(start)
    node=start 
    while(node!=end):
        neighbors = findNeighbors(matrix,node)
        node=neighbors[rd.randint(0,len(neighbors)-1)]
        if node not in visited_nodes:
            visited_nodes.append(node) 
            path.append(node)
    return path


img_path="/content/64room_000.png" #UPDATE
matrix=imageToMatrix(img_path)

 

def generateNeighboringPath(matrix,path): 
    x=rd.randint(0,len(path)-2)
    neighbors=findNeighbors(matrix,path[x])
    output=path.copy()
    output[x]=neighbors[rd.randint(0,len(neighbors)-2)] 
    output[x+1:]=generatePath(matrix,output[x+1],output[-1])
    return output


def cost(path):
    distance=0
    for i in range(len(path)-1):
        distance+=euclidean_distance(path[i],path[i+1])
    return distance


IT = 50      # Initial temperature
FT = 00.1    # Final temperature 
N= 50       # Number of iterations
NT= 50      # Number of iterations per Temperature 
alpha = 00.1 # Geometric Coefficient alpha


def sa(matrix,start,end):  
    CT = IT   # Current temperature
    CP = generatePath(matrix,start,end) # current path (random)
    CL = cost(CP) 
    OP = CP # optimal path
    OL = CL
    for i in range(N):
        print("i= ",i)
        for j in range(NT):
            RP = generateNeighboringPath(matrix,CP)
            RL = cost(RP)
            if RL < CL:
                CP = RP
                CL = RL 
            else: 
                p = math.exp(-(RL-CL)/CT) # Transition Probability 
                r=rd.randint(0,1)
                if r<p:
                    CP = RP
                    CL = RL 
            if CL < OL:
                OP = CP
                OL = CL
        CT *= alpha 
        if CT <= FT:
            break 
    return OP , OL 


start=(10,9)
end=(14,40)

def SA(img_path,start,end,image_name):
    image=imageToMatrix(img_path)
    op , ol = sa(image,start,end) 
    print("optimal path " , op , "path length " , ol)     
    #Draw path in the image and save it
    image=cv.imread(img_path)
    new_image = draw_path(img_path,op) 
    saving_path = '/content/'
    cv.imwrite(os.path.join(saving_path ,image_name), new_image)
    return op, ol 

#TEST

img_path="/content/64room_000.png" #UPDATE
image_name = "64room_000.png"
print(SA(img_path,start,end,"new_"+image_name))    

i=  0
i=  1
i=  2
optimal path  [(10, 9), (11, 9), (12, 8), (13, 7), (14, 6), (13, 5), (14, 5), (15, 5), (12, 5), (16, 5), (17, 6), (18, 6), (17, 5), (16, 6), (18, 5), (15, 6), (15, 7), (17, 7), (18, 8), (19, 8), (19, 7), (17, 8), (16, 9), (16, 10), (15, 10), (15, 9), (15, 11), (16, 12), (17, 12), (17, 11), (17, 10), (16, 8), (15, 8), (14, 7), (14, 8), (14, 9), (13, 10), (12, 9), (11, 10), (12, 10), (12, 11), (13, 11), (12, 12), (12, 13), (11, 13), (11, 12), (10, 12), (9, 12), (9, 13), (9, 14), (10, 14), (11, 15), (11, 16), (10, 15), (9, 16), (9, 15), (8, 16), (10, 16), (11, 17), (12, 17), (12, 18), (12, 18), (11, 19), (11, 18), (10, 18), (9, 17), (8, 17), (7, 16), (6, 15), (6, 14), (6, 13), (7, 12), (6, 12), (8, 13), (9, 14), (9, 13), (9, 12), (8, 14), (5, 12), (5, 11), (7, 14), (7, 15), (8, 16), (7, 17), (7, 18), (9, 16), (9, 15), (10, 15), (10, 13), (11, 14), (12, 13), (13, 13), (12, 14), (11, 13), (10, 17), (11, 17), (12, 17), (11, 16), (9, 18), (8, 18), (7, 20), (8, 20), (9, 21), 